In [2]:
import lxml.html
import requests

response = requests.get("https://gihyo.jp/dp")
root = lxml.html.fromstring(response.content)
root.make_links_absolute(response.url)
urls = root.cssselect('#listBook a[itemprop="url"]')
names = root.cssselect('#listBook p[itemprop="name"]')

for url, name in zip(urls, names):
    u = url.get("href")
    n = name.text_content()
    print(u)
    print(n)
    print("\n")

https://gihyo.jp/dp/ebook/2018/978-4-297-10109-1
基礎入門 ブレッドボードではじめる マイコンプログラミング


https://gihyo.jp/dp/ebook/2018/978-4-297-10013-1
大きな字でわかりやすい 大きな字でわかりやすいWindows 10 インターネット入門


https://gihyo.jp/dp/ebook/2018/978-4-297-10021-6
Bootstrap 4 フロントエンド開発の教科書


https://gihyo.jp/dp/ebook/2018/978-4-297-10034-6
Docker/Kubernetes 実践コンテナ開発入門


https://gihyo.jp/dp/ebook/2018/978-4-7741-9944-3
WEB+DB PRESS Vol.106


https://gihyo.jp/dp/ebook/2018/978-4-7741-9891-0
世界一わかりやすい逆引き事典 世界一わかりやすい Illustrator 逆引き事典 CC対応


https://gihyo.jp/dp/ebook/2018/978-4-7741-9889-7
世界一わかりやすい逆引き事典 世界一わかりやすい Photoshop 逆引き事典 CC対応


https://gihyo.jp/dp/ebook/2018/978-4-7741-9885-9
ゼロからはじめる ゼロからはじめるAmazon Echo スマートガイド


https://gihyo.jp/dp/ebook/2018/978-4-7741-9883-5
今すぐ使えるかんたん 今すぐ使えるかんたん自作パソコン Windows 10対応版［改訂2版］


https://gihyo.jp/dp/ebook/2018/978-4-297-10036-0
OSSライセンスの教科書


https://gihyo.jp/dp/ebook/2018/978-4-297-10053-7
もっと速く，快適に！ Wi-Fiを使いこなす本


https://gihyo.jp/dp/ebook/2018/978-4-297-10058-2
ゼロからはじめる ゼロからはじめるドコモ Xpe

In [10]:
import lxml.html
import requests
import re
from pymongo import MongoClient
import time

In [43]:
def main():
    VezelURL = 'https://www.carsensor.net/usedcar/bHO/s101/optREP1/index.html'
    total_list_num = get_lastpage_num(VezelURL)
    
    client = MongoClient()
    db = client['CARSENSOR']
    collection = db['HONDA_vezel']
    collection.delete_many({})
    
    for i in range(1, total_list_num + 1):
        listpage_url = VezelURL.replace('index', 'index' + str(59))
        urls = scrape_list_page(listpage_url)
        collect_insert(urls, collection)
        break


In [44]:
def collect_insert(urls, collection):
    """
    指定したリストページ内の詳細ページをスクレイピングし、DBに格納
    input:
        - urls: generator object, 詳細ページのurlを生成するgenerator
        - collection: mongodb collection object
    output: None
    """
    for url in urls:
        key = extract_key(url)
        print(key)
        key_check = collection.find_one({'key': key})
        if not key_check:
            usedcar = scrape_detail_page(url, key)
            print(usedcar)
#             collection.insert_one(usedcar)
            time.sleep(1)

In [37]:
def get_lastpage_num(url):
    """
    リストページのhtmlからリストページの総数を取得して返す
    intput:
        - url: str
    output:
        - last_num: int, リストページ数
    """
    response = requests.get(url)
    root = lxml.html.fromstring(response.content)
    # 最後のlistページのurlを取得する
    last_page = root.cssselect('#js-resultBar > div.resultBar__link > div > div.pager__text > a:nth-child(12)')[0]
    # urlから最後のindex番号を取得する
    last_index = last_page.get('href').split("/")[-1]
    m = re.search('index([0-9]+)', last_index)
    # re.search()でマッチしなかった場合のエラー処理
    if m is None:
        raise ValueError('index.html of last page is not found')
    last_num = int(m.group(1))
    return last_num

In [38]:
def scrape_list_page(url):
    """
    リストページから詳細ページのurlを取得し、生成する(generator object)
    input:
        - url: str, リストページのurl
    output:
        generator object, 詳細ページのurlを生成する
    """
    # webページ読み込み、response.contentから文字列を取得してパース
    response = requests.get(url)
    root = lxml.html.fromstring(response.content)
    # 全てのurlを絶対urlに変換
    root.make_links_absolute(response.url)
    htmlElems = root.cssselect('#carList h3[class="casetMedia__body__title"] > a')
    for elem in htmlElems:
        url = elem.get('href')
        yield url

In [39]:
def scrape_detail_page(url, key):
    """
    詳細ページから各種情報を取得し、dictで返す
    input:
        - url: str, 詳細ページのurl
    output:
        - usedcar: dict, dbに格納するドキュメント
    """
    response = requests.get(url)
    root = lxml.html.fromstring(response.content)
    # 本体価格
    base_price = root.cssselect('body > div.page > div:nth-child(6) > main > section > div > div.column__sub > div.priceWrap > div.basePrice > p.basePrice__price')
    base_price = base_price[0].get('content').replace(',', '')
    base_price = int(base_price)
    # 支払総額
    total_price = root.cssselect('body > div.page > div:nth-child(6) > main > section > div > div.column__sub > div.priceWrap > div.totalPrice > p.totalPrice__price')
    total_price = total_price[0].text_content()
    total_price = re.sub(r'\n(\t)+', '', total_price).replace('万円', '')
    try:
        total_price = int(float(total_price) * 10000)
    except ValueError:
        total_price = None
    # 年式
    model_year = root.cssselect('body > div.page > div:nth-child(6) > main > section > div > div.column__sub > div.specWrap > div:nth-child(1) > p.specWrap__box__num')
    model_year = model_year[0].text
    model_year = int(model_year)
    # 走行距離
    distance = root.cssselect('body > div.page > div:nth-child(6) > main > section > div > div.column__sub > div.specWrap > div:nth-child(2)')
    distance = distance[0].text_content()
    distance = re.sub(r'\n(\t)+', '', distance).strip().replace('走行距離', '')
    if '万' in distance:
        distance = distance.replace('万km', '')
        distance = float(distance)
    else:
        distance = distance.replace('km', '')
        try:
            distance = float(distance) / 10000
        except ValueError:
            distance = None
#     distance = float(distance)
    # 車検
    inspection = root.cssselect('body > div.page > div:nth-child(6) > main > section > div > div.column__sub > div.specWrap > div:nth-child(3)')
    inspection = inspection[0].text_content()
    inspection = re.sub(r'\n(\t)+', '', inspection).replace('車検有無', '')
    # 修復歴
    repare = root.cssselect('body > div.page > div:nth-child(6) > main > section > div > div.column__sub > div.specWrap > div:nth-child(4) > p:nth-child(2)')
    repare = repare[0].text
    # 本体情報等
    info = root.cssselect('body > div.page > div:nth-child(6) > main > section > h2 > span')
    info = info[0].text
    info = info.replace('\u3000', ' ').replace('\xa0', ' ')
    # リサイクル料
    recycle = root.cssselect('body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(2) > div > table > tbody > tr:nth-child(6) > td:nth-child(2)')
    recycle = recycle[0].text
    # 法定整備
    legal_maintenance = root.cssselect('body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(2) > div > table > tbody > tr:nth-child(9) > td > p')
    legal_maintenance = legal_maintenance[0].text
    # 保証
    warranty = root.cssselect('body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(2) > div > table > tbody > tr:nth-child(10) > td > p')
    warranty = warranty[0].text
    warranty = warranty.replace('\u3000', ' ')
    # ワンオーナー
    one_owner = root.cssselect('body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(2) > div > table > tbody > tr:nth-child(1) > td:nth-child(4)')
    one_owner = one_owner[0].text
    if one_owner == '◯':
        one_owner = 1
    else:
        one_owner = 0
    # 定期点検記録母
    record_book = root.cssselect('body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(2) > div > table > tbody > tr:nth-child(4) > td:nth-child(2)')
    record_book = record_book[0].text
    if record_book == '◯':
        record_book = 1
    else:
        record_book = 0
    # 禁煙車
    no_smoke = root.cssselect('body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(2) > div > table > tbody > tr:nth-child(5) > td:nth-child(2)')
    no_smoke = no_smoke[0].text
    if no_smoke == '◯':
        no_smoke = 1
    else:
        no_smoke = 0

    region = root.cssselect('body > div.page > div:nth-child(6) > main > section > div > div.column__sub > div.specWrap > div:nth-child(5) > p:nth-child(2)')
    region = region[0].text

    usedcar = {
        'url': response.url,
        'base_price': base_price,
        'total_price': total_price,
        'model_year': model_year,
        'distance': distance,
        'inspection': inspection,
        'info': normalize_spaces(info),
        'recycle': recycle,
        'legal_maintenance': legal_maintenance,
        'warranty': warranty,
        'one_owner': one_owner,
        'record_book': record_book,
        'no_smoke': no_smoke,
        'key': key,
        'region': region
    }
    return usedcar

In [40]:
def normalize_spaces(s):
    """
    文字列内の連続する空白を一つにし、前後の空白を削除する
    input:
        - s: str
    output: str
    """
    return re.sub(r'\s+', ' ', s).strip()

In [41]:
def extract_key(url):
    """
    詳細ページのurlからキーを取得し、返す
    input:
        - url: str, 詳細ページのurl
    output: str
    """
    key = url.split('/')[-2]
    if not key.startswith('CU'):
        raise ValueError('key extract error, detail url is wrong')
    return key

In [42]:
main()

https://www.carsensor.net/usedcar/bHO/s101/optREP1/index59.html
CU8163335937
{'url': 'https://www.carsensor.net/usedcar/detail/CU8163335937/index.html?TRCD=200002', 'base_price': 1768000, 'total_price': None, 'model_year': 2016, 'distance': 1.1, 'inspection': '2019(H31)年11月', 'info': '1.5 X ホンダセンシング 純正ナビ バックカメラ （ブラックメタ）', 'recycle': 'リ済別', 'legal_maintenance': '法定整備無', 'warranty': '保証付：販売店保証 保証期間：3ヵ月 保証距離：無制限', 'one_owner': 1, 'record_book': 0, 'no_smoke': 0, 'key': 'CU8163335937', 'region': '東京都'}
CU8620974004
{'url': 'https://www.carsensor.net/usedcar/detail/CU8620974004/index.html?TRCD=200002', 'base_price': 2259000, 'total_price': None, 'model_year': 2016, 'distance': 1.2, 'inspection': '車検整備別', 'info': '1.5 ハイブリッド Z ホンダセンシング 純正SDナビ （クリスタルブラック・パール）', 'recycle': 'リ済別', 'legal_maintenance': '法定整備別\u3000費用：39,636円', 'warranty': '保証付：販売店保証 保証期間：3ヵ月 保証距離：3,000km', 'one_owner': 0, 'record_book': 0, 'no_smoke': 0, 'key': 'CU8620974004', 'region': '愛知県'}
CU8368651075
{'url': 'https://www.c

CU6702476362
{'url': 'https://www.carsensor.net/usedcar/detail/CU6702476362/index.html?TRCD=200002', 'base_price': 2390000, 'total_price': None, 'model_year': 2016, 'distance': 0.8, 'inspection': '2019(H31)年8月', 'info': '1.5 ハイブリッド RS ホンダセンシング 純正ナビ （ブラック）', 'recycle': 'リ済別', 'legal_maintenance': '法定整備無', 'warranty': '保証無', 'one_owner': 0, 'record_book': 0, 'no_smoke': 0, 'key': 'CU6702476362', 'region': '福岡県'}
CU7293083933
{'url': 'https://www.carsensor.net/usedcar/detail/CU7293083933/index.html?TRCD=200002', 'base_price': 2398000, 'total_price': None, 'model_year': 2016, 'distance': 1.8, 'inspection': '2019(H31)年8月', 'info': '1.5 ハイブリッド Z シティブレーキ ナビ/TV （ホワイトオーキッド・パール）', 'recycle': 'リ済別', 'legal_maintenance': '法定整備別\u3000費用：38,800円', 'warranty': '保証付：販売店保証 保証期間：1ヵ月 保証距離：1,000km', 'one_owner': 0, 'record_book': 1, 'no_smoke': 0, 'key': 'CU7293083933', 'region': '熊本県'}
CU8546052371
{'url': 'https://www.carsensor.net/usedcar/detail/CU8546052371/index.html?TRCD=200002', 'base_price': 22030

In [90]:
a = float('---')
a *= 10000
int(a)

ValueError: could not convert string to float: '---'

In [22]:
t = '\n\t\t\t\t\t\t\t\t車検有無\n\t\t\t\t\t\t\t\t2021(H33)\n\t\t\t\t\t\t\t\t年7月\n\t\t\t\t\t\t\t'
print(t)
t2 = re.sub(r'\n(\t)+', '', t)
t2.replace('車検有無', '')


								車検有無
								2021(H33)
								年7月
							


'2021(H33)年7月'

In [1]:
from pymongo import MongoClient
client = MongoClient()

In [2]:
db = client.test

In [3]:
collection = db.spots

In [4]:
collection.insert_one({'name': 'kojiro tanaka', 'age': '25'})

In [5]:
data = [
    {'name': 'tomomi yoneda', 'age': '24'},
    {'name': 'hiroki tanaka', 'age': '30'}
]
collection.insert_many(data)

In [6]:
for person in collection.find():
    print(person)

{'_id': ObjectId('5b80b844b2bf2449704ef3e1'), 'name': 'kojiro tanaka', 'age': '25'}
{'_id': ObjectId('5b80b8c2b2bf2449704ef3e2'), 'name': 'tomomi yoneda', 'age': '24'}
{'_id': ObjectId('5b80b8c2b2bf2449704ef3e3'), 'name': 'hiroki tanaka', 'age': '30'}


In [7]:
for person in collection.find({'name': 'kojiro tanaka'}):
    print(person)

{'_id': ObjectId('5b80b844b2bf2449704ef3e1'), 'name': 'kojiro tanaka', 'age': '25'}
